# Nexstrain Processing

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import re
from pathlib import Path

# ----- constants -----
IN_PATH = Path("ns_metadata_100k.tsv")   # adjust if needed
OUT_PATH = Path("ns_mutated_spike_100k.fasta")

# User-provided WT Spike segment
WT = "SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG"

# Positions covered by WT in Wuhan-Hu-1 Spike numbering
ABS_START = 349
ABS_END = 348 + len(WT)  # inclusive

# ----- helper functions -----
def get_sequence(WT_seq: str, site_abs: int, mutation_to: str, wt_aa: str) -> str:
    """Apply single substitution after checking WT residue."""
    site = site_abs - ABS_START
    if mutation_to=='*':
        return WT_seq
    assert WT_seq[site] == wt_aa, f"WT residue mismatch at {site_abs}: found {WT_seq[site]}, expected {wt_aa}"
    return WT_seq[:site] + mutation_to + WT_seq[site + 1:]

# Parse tokens like "S:R158G" and return (from_aa, pos_abs, to_aa) or None
_mut_re = re.compile(r"^S:([A-Z\*])(\d+)([A-Z\*])$")

def parse_spike_sub(token: str):
    token = token.strip()
    m = _mut_re.match(token)
    if not m:
        return None
    f, pos, t = m.group(1), int(m.group(2)), m.group(3)
    return f, pos, t

def apply_spike_mutations(WT_seq: str, aa_substitutions: str) -> tuple[str, list[str]]:
    """
    Keep S: substitutions whose absolute site lies in [ABS_START, ABS_END].
    Apply only if current residue matches the 'from' amino acid.
    """
    if not isinstance(aa_substitutions, str) or not aa_substitutions.strip():
        return WT_seq, []

    tokens = [t.strip() for t in re.split(r"[,\s]+", aa_substitutions) if t.strip()]
    applied = []
    seq = WT_seq

    for tok in tokens:
        if not tok.startswith("S:"):
            continue
        parsed = parse_spike_sub(tok)
        if not parsed:
            continue
        from_aa, pos_abs, to_aa = parsed

        # within window
        if pos_abs < ABS_START or pos_abs > ABS_END:
            continue

        idx = pos_abs - ABS_START
        if idx < 0 or idx >= len(seq):
            continue

        # compatible?
        if seq[idx] != from_aa:
            continue

        # apply with WT check
        seq = get_sequence(seq, pos_abs, to_aa, from_aa)
        applied.append(tok)

    return seq, applied

def fasta_header(date, region, lineage):
    d = "" if pd.isna(date) else str(date)
    r = "" if pd.isna(region) else str(region)
    p = "" if pd.isna(lineage) else str(lineage)
    return f">{d}|{r}|{p}"

# ----- main -----
def main():
    df = pd.read_csv(IN_PATH, sep="\t", dtype=str, keep_default_na=True, na_values=["", "NA", "NaN"])

    required = ["date", "region", "pango_lineage", "aaSubstitutions"]
    for col in required:
        if col not in df.columns:
            df[col] = pd.NA

    out_lines = []
    for _, row in df.iterrows():
        date = row["date"]
        region = row["region"]
        lineage = row["pango_lineage"]
        aa_subs = row["aaSubstitutions"]

        mutated_seq, _ = apply_spike_mutations(WT, aa_subs)

        header = fasta_header(date, region, lineage)
        out_lines.append(header)
        out_lines.append(mutated_seq)

    OUT_PATH.write_text("\n".join(out_lines) + "\n", encoding="utf-8")
    print(f"Wrote {len(out_lines)//2} sequences to {OUT_PATH.resolve()}")

if __name__ == "__main__":
    main()


Wrote 99773 sequences to E:\ESCAPE_MAP_DRAFT\seq_data\ns_mutated_spike.fasta
